<a href="https://colab.research.google.com/github/bksat90/ImageSegmentation/blob/main/Image_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation of KerasCV.
!pip install -q --upgrade keras-cv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 12.6 MB/s eta 0:00:00


In [ ]:
# import libraries
import random
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import keras
import keras_cv

from keras import layers
from keras_cv import bounding_box
from keras_cv import visualization
from pycocotools.coco import COCO

%matplotlib inline

Using TensorFlow backend
